<h1>Creating Digital Audio</h1>

With Python, we can create, record, and manipulate audio with just a few lines of code. After going through this notebook, you'll understand some of the basics of digital audio, and how we can interact with audio through code. Also, you'll see how powerful Python is, especially if we use some of the many available libraries.

A basic building block of audio is a sine tone. Actually, all sounds we hear can be modeled as a sum of different sine tones. So let's begin by creating a sine wave, listening to it, and looking at with different visualizations.

The main characteristic of a sine tone is it's _frequency_. The frequency sets the pitch of the sound - how high or low it sounds. The range of frequencies for human hearing is 20-20,000 Hz. What's a good frequency for our sine tone that's not too high or too low?

Think of an orchestra or symphony tuning together at the start of a performance. This is a fairly pleasing frequency.

In [ ]:
from IPython.display import HTML, Audio, display, clear_output
%matplotlib inline
HTML('<iframe width="800" height="600" src="https://www.youtube.com/embed/KfSH1ezevjM?rel=0" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>')

The tone they're tuning to is "A 440". It's the note A, at 440 _Hertz_. We can generate a digital version of this tone with a few lines of Python.

A tone is just a sine wave, following the formula

$$
wave = \sin(2 * pi * frequency * time)
$$

For digital audio, we generate samples at even time steps. Samples are the smallest unit of a digital audio wave, similar to how a pixel is the smallest unit in a digital image.  We need to set a sampling rate, or how often to create digital samples.

This image shows an example of a digital audio signal. The red line is the original audio, and the blue dots are the digital representation. Sampling rate is how often to generate those blue dots.
![Digital Audio](img/samples.svg)

Let's set the sampling rate to 48,000 samples per second (also known as _Hertz_ or Hz), fast enough to capture all sounds in the range of human hearing. We could generate other very low or high pitches with this sampling rate.

The lines of code below set up some variables with our parameters, like the tone's frequency (440 Hz to match the orchestra sound), duration, and sample rate. The sample rate is used to generate a timeline, $t$. It will be a list of numbers representing the clock time for each sample. It starts at time zero, the next number is $T$, then $2*T$, $3*T$, and so on.

To help us make this list, we import numpy, which is a Python library useful for manipulating numbers, vectors, and matrices in Python. numpy makes this timeline for us automatically, if we give it the start value, the end value, and the number of steps to take inbetween.

In [ ]:
tone_frequency = 440 # Hz
tone_duration = 4    # seconds

samplerate = 48000 # Hz.   Sample period T = 1/samplerate
print ("One digital audio sample lasts just {:.10f} seconds!".format(1/samplerate))

total_samples = tone_duration*samplerate+1
print ("We'll generate {} total samples".format(total_samples))

import numpy as np
t = np.linspace(0, tone_duration, total_samples)
print("Our list of sample times: {}".format(t))

Finally let's make the actual wave. Recall our formula...

$$
wave = \sin(2 * pi * frequency * time)
$$

numpy helps us again by providing the sine function and the constant Pi.

In [ ]:
wave = np.sin( 2 * np.pi * tone_frequency * t)

We've now created our wave! How can we check it out? Here's a function for listening to the audio, as well as visualizing it in time and frequency.

In [ ]:
def view_audio(audio, Fs, min_sample=0, max_sample=-1):
    
    import matplotlib.pyplot as plt
    from matplotlib import rcParams
    rcParams.update({'font.size': 20})
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10))
    plt.subplots_adjust(hspace=0.37, top=0.95, bottom=0.1)
    plot_audio = audio[min_sample:max_sample].squeeze()
    ax1.plot(plot_audio, '.', ms='2')
    ax1.set_title('Audio Samples')
    ax1.set_xlabel('Sample number'); ax1.set_ylabel('Sample amplitude')
     
    ax2.specgram(plot_audio, NFFT=1024, Fs=Fs, noverlap=256);
    ax2.set_yscale('log')
    ax2.set_ylim(10,20000)
    ax2.set_title('Spectrogram')
    ax2.set_xlabel('Time (sec)'); ax2.set_ylabel('Frequency (Hz)')
    
    display(Audio(audio.squeeze(), rate=samplerate))
    

When you listen to the audio - does it sound similar to the orchestra clip? How is it different?

In [ ]:
view_audio(wave, samplerate, min_sample=0, max_sample=2048)

Another interesting basic signal is a "chirp". We can sweep through some frequencies with a the wave, and the spectrogram shows us how the frequency changes over time.

In [ ]:
start_freq = 4  # This might be below human hearing, but it helps to visualize in the plot below
end_freq = 8000 # If you go too high the chirp will get painful to listen to towards the end!
freq_sweep = np.logspace(np.log2(start_freq), np.log2(end_freq), num=total_samples, base=2)
chirp_wave = np.sin( np.cumsum(2 * np.pi * freq_sweep/samplerate))
view_audio(chirp_wave, samplerate)

<h1>Recording Audio</h1>

These waves are easy to make in code, but they're pretty boring to listen to. Natural sounds you hear every day are more complex. Let's take a look at sounds recorded from a microphone. There's a Python library for recording and playing audio called sounddevice. First we'll check out the devices available to the computer for recording audio.

In [ ]:
import sounddevice as sd
print(sd.query_devices())

Now we'll run the recording, giving the function info about duration, sample rate, and device

In [ ]:
recording_duration = 5  # seconds
my_recording = sd.rec(recording_duration*samplerate, samplerate=samplerate, device=0, channels=1)
sd.wait()

In [ ]:
view_audio(my_recording, samplerate)

What do you notice about the plots of the samples and the spectrogram? Do they look like what you would expect?
The _overtones_ you can see in a voice or instrument playing are part of the audio "fingerprint" of that particular sound source.

<h1>Reading Audio From a File</h1>

Python can also read sounds from a file. The scipy package helps us out. Below I'm loading a file from my computer, it will not be distributed with the code, but you can try another file on your computer.

In [ ]:
from scipy.io.wavfile import read as wavread
file_Fs, file_wav = wavread('audio/hahn-bach-mono.wav')
view_audio(file_wav, Fs=file_Fs)

<h1>Changing Audio</h1>

Now let's use code to change the way audio sounds.

One way to change audio is to filter out parts of the spectrum - we can attenuate some of the frequencies.

We'll use the Python library scipy to calculate the filters for us. By changing the parameters of the filter, we can change how the filtered audio sounds. One of the most drastic changes is if we filter out only low frequencies, compared to only high frequencies. Try  swapping between low_filter and high_filter in the lfilter() function below!

In [ ]:
from scipy.signal import iirdesign, lfilter
# iirdesign(wp, ws, gpass, gstop, analog=False, ftype='ellip', output='ba')
# examples:
# lowpass = iirdesign(0.05, 0.1, 1, 60)
# highpass = iirdesign(0.1, 0.05, 1, 60)
cutoff_frequency = 700 # Hz
filt_f_lo = cutoff_frequency/file_Fs*2
filt_f_hi = cutoff_frequency/file_Fs*2 + 0.01
print("Filter parameters: {:.5f} {:.5f}".format(filt_f_lo, filt_f_hi))
low_filter = iirdesign(filt_f_lo, filt_f_hi, 1, 80)
high_filter = iirdesign(filt_f_hi, filt_f_lo, 1, 80)

filtered_audio = lfilter(*low_filter, file_wav.squeeze())
view_audio(filtered_audio, Fs=file_Fs)

<h1>Try it out!</h1>

What other things can you imagine to manipulate sound?
- Add sine waves together - different frequencies, different amplitudes
- Change the sample rate when recording
- Change the sample rate when playing back
- Use the filter functions with different parameters
- ....?

<h1>References</h1>
- digital audio - https://en.wikipedia.org/wiki/Digital_audio
- numpy - https://docs.scipy.org/doc/numpy/reference/routines.html
- scipy - https://docs.scipy.org/doc/scipy/reference/index.html
- matplotlib pyplot - https://matplotlib.org/api/_as_gen/matplotlib.pyplot.html
- sounddevice - https://python-sounddevice.readthedocs.io/


In [9]:
# here's some images to show the impact of sampling rate
# the images with more blue dots are double the sampling rate of the others
# a higher sampling rate tracks the wave with better fidelity
slideshow = """
<img></img> <!- this is a dummy img to avoid changes in alignment from jupyter ->
<img class="audioSampleSlides" src="img/samples.svg" onClick="nextImg()">
<img class="audioSampleSlides" src="img/digital-only.svg" onClick="nextImg()">
<img class="audioSampleSlides" src="img/samples-hires.svg" onClick="nextImg()">
<img class="audioSampleSlides" src="img/digital-only-hires.svg" onClick="nextImg()">
<script>
    var slideIndex=1;
    showSlide(slideIndex);
    function nextImg() {
        showSlide(slideIndex += 1);
    }
    function showSlide(n) {
        var i;
        var x = document.getElementsByClassName("audioSampleSlides");
        if (n > x.length) {slideIndex = 1};
        if (n < 1) {slideIndex = x.length};
        for (i = 0; i < x.length; i++) {
            x[i].style.display = "none"; 
        }
        x[slideIndex-1].style.display = "block"; 
    }
</script>
"""
from IPython.display import display, HTML
display(HTML(slideshow))

In [ ]:
# HACK: use saved file as recorded audio, in case recording doesn't work well
from scipy.io.wavfile import read as wavread
file_Fs, file_wav = wavread('audio/scale.wav')
my_recording = file_wav